In [2]:
import numpy as np
import time
import scipy as sp
import scipy.linalg
from scipy import sparse
import scipy.sparse.linalg
import random
import matplotlib.pyplot as plt
import warnings

In [3]:
def prep_data():
    x = np.loadtxt("simulated_genos", delimiter=" ", dtype="float32")
    y = np.array([[1] * 10000 + [0] * 10000], dtype="float32")
    y_c = y - 0.5
    return x, y, y_c

x, y, y_c = prep_data()

In [4]:
n = 20000
X = x[:,:]
print("X shape:", X.shape)
print("Rank of X:", np.linalg.matrix_rank(X))
print ("Number of 1:",np.sum(x))
n = X.shape[0]
p = X.shape[1]

X shape: (20000, 50)
Rank of X: 50
Number of 1: 953.0


Convert X to CSR format

In [5]:
X_sparse = sp.sparse.csr_matrix(X)

# SVD

Use sp.linalg.svdvals to calculate singular value

In [6]:
start_time = time.clock()
for i in range(10):
    np_SVD_result = sp.linalg.svdvals(X[:n,:])
end_time = time.clock()
print("np SVD time", end_time - start_time)
print("X SV:", np_SVD_result, len(np_SVD_result))
print("X Eigen:",np_SVD_result ** 2)
print("Mean:", sum(np_SVD_result ** 2)/4)

np SVD time 0.3282119132962503
X SV: [6.324555  6.171154  6.091167  6.0882163 6.0827622 5.7486973 5.5825267
 5.3971276 5.3917594 5.388871  5.385164  5.2915025 5.258997  5.102881
 5.052542  5.0031204 4.9999995 4.898985  4.8989787 4.6904163 4.683336
 4.582577  4.5481615 4.472136  4.472136  4.4501877 4.2222996 4.116715
 4.0000014 4.        3.863798  3.6055512 3.4641018 3.4572375 3.3166246
 3.316624  3.1622791 3.162278  3.1622767 3.0000017 2.8284268 2.6457512
 2.6457512 2.6457512 2.4494898 2.2360685 2.        2.        1.9899907
 1.       ] 50
X Eigen: [39.999996  38.08314   37.102314  37.06638   36.999996  33.04752
 31.164604  29.128986  29.07107   29.039932  28.999989  27.999998
 27.65705   26.039394  25.528183  25.031214  24.999996  24.000053
 23.999992  22.000006  21.933634  21.000013  20.685774  20.
 20.        19.80417   17.827814  16.947342  16.000011  16.
 14.928934  13.        12.000001  11.952491  10.999999  10.999994
 10.00001   10.000002   9.999994   9.0000105  7.9999986  6.999

Use sp.sparse.linalg.svds to calculate singular value

In [7]:
start_time = time.clock()
for i in range(10):
    sp_SVD_result = sparse.linalg.svds(X_sparse, k = 49)
end_time = time.clock()
print("sp sparse SVD time", end_time - start_time)
print("X_sparse SV:", sp_SVD_result[1])

sp sparse SVD time 0.7978172595979696
X_sparse SV: [1.9899871 1.9999968 2.000001  2.236067  2.4494889 2.6457508 2.6457512
 2.6457515 2.8284268 2.999999  3.1622775 3.1622777 3.1622777 3.3166242
 3.3166244 3.4572372 3.4641016 3.6055515 3.8637974 3.9999995 4.0000005
 4.116715  4.2223005 4.4501877 4.472136  4.4721365 4.5481615 4.582576
 4.683335  4.6904154 4.89898   4.89898   5.        5.003121  5.0525436
 5.102881  5.258997  5.291503  5.3851643 5.3888717 5.39176   5.3971276
 5.5825267 5.7486978 6.0827622 6.088215  6.091167  6.1711535 6.324554 ]


In [8]:
print(sorted((sp_SVD_result[1])**2, reverse = True))

[39.999985, 38.083138, 37.102314, 37.06636, 36.999996, 33.047527, 31.164604, 29.128986, 29.071075, 29.039938, 28.999994, 28.000004, 27.65705, 26.039394, 25.528196, 25.03122, 25.0, 24.000006, 24.000006, 21.999996, 21.933624, 21.0, 20.685774, 20.000006, 20.0, 19.80417, 17.827822, 16.947342, 16.000004, 15.999996, 14.92893, 13.000002, 12.0, 11.952489, 10.999997, 10.999996, 10.0, 10.0, 9.999999, 8.999994, 7.9999986, 7.000001, 6.9999995, 6.999997, 5.9999957, 4.9999957, 4.000004, 3.9999871, 3.960049]


# Eigenvalue

This standard implementation of scipy.sparse.eigvals is a wrapper of the software [**ARPACK**](http://www.caam.rice.edu/software/ARPACK/
).
The method is Implicitly Restarted Arnoldi Methods. See detail of this in [ARPACK User Guide](http://li.mit.edu/Archive/Activities/Archive/CourseWork/Ju_Li/MITCourses/18.335/Doc/ARPACK/Lehoucq97.pdf).

Calculate $XX^T$

In [9]:
XX_t = X  @ X.T
#np.linalg.matrix_rank(XX_t)

In [1]:
type(XX_t)
print(XX_t.shape)
print(XX_t)
print("Sum of XX^T", np.sum(XX_t))

NameError: name 'XX_t' is not defined

Use scipy.linalg.eigvals to calculate eigenvalue

In [22]:
start_time = time.clock()
ev = sp.linalg.eigvals(XX_t)
end_time = time.clock()
print("numpy Eigen Time:", end_time - start_time)

In [23]:
ev = sorted(np.real(ev), reverse=True)

numpy Eigen Time: 89.63207738364702


In [24]:
print(ev[:50])
sum(ev[:50])/4

[40.00016, 38.08318, 37.102364, 37.06637, 37.000046, 33.047585, 31.164608, 29.128963, 29.071026, 29.04003, 29.000072, 27.999949, 27.657042, 26.039454, 25.52822, 25.03121, 25.000042, 24.000017, 24.000011, 22.000015, 21.933619, 21.000021, 20.685772, 20.000021, 20.00002, 19.804165, 17.827856, 16.947296, 16.000011, 15.999988, 14.928966, 13.000019, 12.000004, 11.952526, 11.000026, 11.000006, 9.999998, 9.99999, 9.999983, 9.000001, 7.9999933, 7.0000095, 7.0000076, 7.0, 6.000024, 5.0000033, 3.999998, 3.9999948, 3.9600687, 1.0]


238.25018620491028

Use sp.sparse.linalg.eigs to calculate eigenvalue

In [13]:
XX_t_sparse = X_sparse @ X_sparse.T
sp.rank(XX_t_sparse)
start_time = time.clock()
ev_sparse = sparse.linalg.eigs(XX_t_sparse, k=50)
end_time = time.clock()
print("Sparse Eigen Time:", end_time - start_time)
ev_sparse = np.real(ev_sparse[0])
print(sorted(ev_sparse, reverse=True))

C:\Users\songdongyuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  


Sparse Eigen Time: 0.4209739003844817
[40.00009, 38.0832, 37.102425, 37.066383, 36.999996, 33.047543, 31.164667, 29.12903, 29.071125, 29.039948, 29.0, 28.000029, 27.657076, 26.039392, 25.528198, 25.031212, 25.000002, 24.000017, 24.000006, 22.000006, 21.933622, 20.999989, 20.685759, 20.0, 19.999977, 19.804165, 17.827816, 16.947361, 16.00001, 15.9999895, 14.928922, 12.999985, 12.000008, 11.952486, 10.999998, 10.99999, 10.000014, 10.0, 9.999995, 9.000008, 8.000005, 7.0000014, 7.0, 6.9999967, 5.999999, 5.0000052, 4.0000024, 4.0000005, 3.9600654, 0.9999981]


In [25]:
sum(ev_sparse / 4)

238.2501277923584

# irlbpy (SVD)

Try to implement **irlbpy**

[IRLB method](https://epubs.siam.org/doi/abs/10.1137/04060593X)

In [26]:
# Matrix-vector product wrapper
# A is a numpy 2d array or matrix, or a scipy matrix or sparse matrix.
# x is a numpy vector only.
# Compute A.dot(x) if t is False,
#         A.transpose().dot(x)  otherwise.
def mult(A,x,t=False):
  if(sparse.issparse(A)):
    m = A.shape[0]
    n = A.shape[1]
    if(t):
      return(sparse.csr_matrix(x).dot(A).transpose().todense().A[:,0])
    return(A.dot(sparse.csr_matrix(x).transpose()).todense().A[:,0])
  if(t):
    return(x.dot(A))
  return(A.dot(x))

def orthog(Y,X):
  """Orthogonalize a vector or matrix Y against the columns of the matrix X.
  This function requires that the column dimension of Y is less than X and
  that Y and X have the same number of rows.
  """
  dotY = mult(X,Y,t=True)
  return (Y - mult(X,dotY))

# Simple utility function used to check linear dependencies during computation:
def invcheck(x):
  eps2  = 2*np.finfo(np.float).eps
  if(x>eps2):
    x = 1/x
  else:
    x = 0
    #warnings.warn("Ill-conditioning encountered, result accuracy may be poor")
  return(x)

def irlb(A,n,tol=0.0001,maxit=50):
  """Estimate a few of the largest singular values and corresponding singular
  vectors of matrix using the implicitly restarted Lanczos bidiagonalization
  method of Baglama and Reichel, see:

  Augmented Implicitly Restarted Lanczos Bidiagonalization Methods,
  J. Baglama and L. Reichel, SIAM J. Sci. Comput. 2005

  Keyword arguments:
  tol   -- An estimation tolerance. Smaller means more accurate estimates.
  maxit -- Maximum number of Lanczos iterations allowed.

  Given an input matrix A of dimension j * k, and an input desired number
  of singular values n, the function returns a tuple X with five entries:

  X[0] A j * nu matrix of estimated left singular vectors.
  X[1] A vector of length nu of estimated singular values.
  X[2] A k * nu matrix of estimated right singular vectors.
  X[3] The number of Lanczos iterations run.
  X[4] The number of matrix-vector products run.

  The algorithm estimates the truncated singular value decomposition:
  A.dot(X[2]) = X[0]*X[1].
  """
  nu     = n
  m      = A.shape[0]
  n      = A.shape[1]
  if(min(m,n)<2):
    raise Exception("The input matrix must be at least 2x2.")
  m_b    = min((nu+20, 3*nu, n))  # Working dimension size
  mprod  = 0
  it     = 0
  j      = 0
  k      = nu
  smax   = 1
  ifsparse = sparse.issparse(A)

  V  = np.zeros((n,m_b))
  W  = np.zeros((m,m_b))
  F  = np.zeros((n,1))
  B  = np.zeros((m_b,m_b))

  V[:,0]  = np.random.randn(n) # Initial vector
  V[:,0]  = V[:,0]/np.linalg.norm(V)

  while(it < maxit):
    if(it>0): j=k
    W[:,j] = mult(A,V[:,j])
    mprod+=1
    if(it>0):
      W[:,j] = orthog(W[:,j],W[:,0:j]) # NB W[:,0:j] selects columns 0,1,...,j-1
    s = np.linalg.norm(W[:,j])
    sinv = invcheck(s)
    W[:,j] = sinv*W[:,j]
    # Lanczos process
    while(j<m_b):
      F = mult(A,W[:,j],t=True)
      mprod+=1
      F = F - s*V[:,j]
      F = orthog(F,V[:,0:j+1])
      fn = np.linalg.norm(F)
      fninv= invcheck(fn)
      F  = fninv * F
      if(j<m_b-1):
        V[:,j+1] = F
        B[j,j] = s
        B[j,j+1] = fn 
        W[:,j+1] = mult(A,V[:,j+1])
        mprod+=1
        # One step of classical Gram-Schmidt...
        W[:,j+1] = W[:,j+1] - fn*W[:,j]
        # ...with full reorthogonalization
        W[:,j+1] = orthog(W[:,j+1],W[:,0:(j+1)])
        s = np.linalg.norm(W[:,j+1])
        sinv = invcheck(s) 
        W[:,j+1] = sinv * W[:,j+1]
      else:
        B[j,j] = s
      j+=1
    # End of Lanczos process
    S    = np.linalg.svd(B)
    R    = fn * S[0][m_b-1,:] # Residuals
    if(it<1):
      smax = S[1][0]  # Largest Ritz value
    else:
      smax = max((S[1][0],smax))

    conv = sum(np.abs(R[0:nu]) < tol*smax)
    if(conv < nu):  # Not coverged yet
      k = max(conv+nu,k)
      k = min(k,m_b-3)
    else:
      break
    # Update the Ritz vectors
    V[:,0:k] = V[:,0:m_b].dot(S[2].transpose()[:,0:k])
    V[:,k] = F 
    B = np.zeros((m_b,m_b))
    # Improve this! There must be better way to assign diagonal...
    for l in range(0,k):
      B[l,l] = S[1][l]
    B[0:k,k] = R[0:k]
    # Update the left approximate singular vectors
    W[:,0:k] = W[:,0:m_b].dot(S[0][:,0:k])
    it+=1

  U = W[:,0:m_b].dot(S[0][:,0:nu])
  V = V[:,0:m_b].dot(S[2].transpose()[:,0:nu])
  return((U,S[1][0:nu],V,it,mprod))

Check the result of **irlbpy**

In [27]:
result_X = irlb(X, 50)

In [28]:
result_X[1]

array([6.32455532, 6.17115224, 6.09116741, 6.08821591, 6.08276253,
       5.74869651, 5.58252703, 5.39712732, 5.39175925, 5.38887151,
       5.38516481, 5.29150262, 5.2589973 , 5.1028813 , 5.05254332,
       5.00312107, 5.        , 4.89897949, 4.89897949, 4.69041576,
       4.68333503, 4.58257569, 4.54816118, 4.47213595, 4.47213595,
       4.45018781, 4.22230026, 4.11671501, 4.        , 4.        ,
       3.86379764, 3.60555128, 3.46410162, 3.45723711, 3.31662479,
       3.31662479, 3.16227766, 3.16227766, 3.16227766, 3.        ,
       2.82842712, 2.64575131, 2.64575131, 2.64575131, 2.44948974,
       2.23606798, 2.        , 2.        , 1.9899909 , 1.        ])

In [29]:
start_time = time.clock()
for i in range(10):
    result_X_sparse = irlb(X_sparse, 50)
end_time = time.clock()
print("Time of irlbpy:", end_time - start_time)
result_X_sparse[1]

Time of irlbpy: 5.598269184738456


array([6.32455532, 6.17115224, 6.09116741, 6.08821591, 6.08276253,
       5.74869651, 5.58252703, 5.39712732, 5.39175925, 5.38887151,
       5.38516481, 5.29150262, 5.2589973 , 5.1028813 , 5.05254332,
       5.00312107, 5.        , 4.89897949, 4.89897949, 4.69041576,
       4.68333503, 4.58257569, 4.54816118, 4.47213595, 4.47213595,
       4.45018781, 4.22230026, 4.11671501, 4.        , 4.        ,
       3.86379764, 3.60555128, 3.46410162, 3.45723711, 3.31662479,
       3.31662479, 3.16227766, 3.16227766, 3.16227766, 3.        ,
       2.82842712, 2.64575131, 2.64575131, 2.64575131, 2.44948974,
       2.23606798, 2.        , 2.        , 1.9899909 , 1.        ])

In [30]:
print(result_X_sparse[1] ** 2)
print(sum(result_X_sparse[1] ** 2)/4)

[40.         38.08311991 37.10232046 37.06637298 37.         33.04751155
 31.16460799 29.12898331 29.07106781 29.0399362  29.         28.
 27.65705259 26.03939761 25.52819398 25.03122049 25.         24.
 24.         22.         21.93362702 21.         20.68577012 20.
 20.         19.80417152 17.82781952 16.9473425  16.         16.
 14.92893219 13.         12.         11.95248845 11.         11.
 10.         10.         10.          9.          8.          7.
  7.          7.          6.          5.          4.          4.
  3.9600638   1.        ]
238.24999999999991


The result is quite fine, although it is time consuming.